# Caderno 5 - Aplica doc2query

## 1. Carrega o modelo e aloca na GPU/CPU

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.to(device)
data = data.to(device)

## 2. Aplica doc2query

Peguei o exemplo direto do site do modelo e adaptei: https://huggingface.co/doc2query/msmarco-portuguese-mt5-base-v1

O ideal era aplicar isso em batch.

In [33]:
def doc2query(texto, random_sampling=True, num_return_sequences=5, top_p=0.95, top_k=10, max_length=64, seed=None):
    input_ids = tokenizer.encode(texto, return_tensors='pt')

    if seed is not None:
        torch.manual_seed(seed)
    
    with torch.no_grad():
        # Here we use top_k / top_k random sampling. It generates more diverse queries, but of lower quality
        if random_sampling:
            outputs = model.generate(
                input_ids=input_ids,
                max_length=max_length,
                do_sample=True,
                top_p=top_p,
                top_k=top_k,
                num_return_sequences=num_return_sequences
            )
        else:
            # Here we use Beam-search. It generates better quality queries, but with less diversity
            outputs = model.generate(
                input_ids=input_ids, 
                max_length=max_length, 
                num_beams=num_return_sequences, 
                no_repeat_ngram_size=2, 
                num_return_sequences=num_return_sequences, 
                early_stopping=True
            )
        
    queries = [tokenizer.decode(out, skip_special_tokens=True) for out in outputs]
    return queries

In [39]:
text = "Não caracteriza marco interruptivo da prescrição das pretensões punitiva e ressarcitória do TCU ato de investigação dos fatos que não contém medidas inequívocas de apuração de condutas individualmente descritas e imputadas ao responsável."
create_queries(text, random_sampling=True, num_return_sequences=5, top_p=0.95, top_k=10, max_length=64, seed=42)


['definição de prescrição tcu',
 'o que é uma prescrição de tcu?',
 'é uma préscrição tcu suspensa e ressarcitória',
 'definição de ato de investigação do tcu',
 'definição de ato interruptivo da prescrição de pretensões punitivas e ressarcitória de tcu']